# HW 8 Vladimir Saraikin

In [1]:
import numpy as  np
import pandas as pd
from sklearn.datasets import load_diabetes, load_wine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from scipy.stats import shapiro, pearsonr
from statsmodels.stats.multitest import multipletests

from scipy import stats

## Task 2

### 1

In [2]:
data = load_diabetes(as_frame=True)
df = data['frame']

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

p_values = []
features = df_train.columns[:-1]

for feature in features:
    if shapiro(df_train[feature]).pvalue < 0.05:
        print(f"Normality assumption not met for {feature}")

    correlation, p_value = pearsonr(df_train[feature], df_train['target'])
    p_values.append(p_value)

rejected, corrected_p_values, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

selected_features = features[rejected]

model = LinearRegression()
model.fit(df_train[selected_features], df_train['target'])

predictions = model.predict(df_test[selected_features])
rmse = np.sqrt(mean_squared_error(df_test['target'], predictions))

print("Selected features:", selected_features)
print("RMSE on testing dataset:", rmse)



Normality assumption not met for age
Normality assumption not met for sex
Normality assumption not met for bmi
Normality assumption not met for bp
Normality assumption not met for s1
Normality assumption not met for s2
Normality assumption not met for s3
Normality assumption not met for s4
Normality assumption not met for s5
Selected features: Index(['age', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], dtype='object')
RMSE on testing dataset: 54.651010257034045


### 2

In [3]:
data = load_diabetes(as_frame=True)
df = data['frame']
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

ridge = Ridge(alpha=1.0)
ridge.fit(df_train.drop(columns=['target']), df_train['target'])

residuals = df_train['target'] - ridge.predict(df_train.drop(columns=['target']))
if shapiro(residuals).pvalue < 0.05:
    print("Normality assumption not met for residuals")

X_train_sm = sm.add_constant(df_train.drop(columns=['target']))  # Adding a constant for OLS
model = sm.OLS(df_train['target'], X_train_sm)
results = model.fit()

conf_intervals = results.conf_int(alpha=0.05)  # 95% CI
print("Confidence Intervals:\n", conf_intervals)

significant_features = conf_intervals.loc[~((conf_intervals[0] <= 0) & (conf_intervals[1] >= 0))].index

significant_features = [feat for feat in significant_features if feat != 'const']
X_train_significant = df_train[significant_features]
X_test_significant = df_test[significant_features]

lin_reg = LinearRegression()
lin_reg.fit(X_train_significant, df_train['target'])

predictions = lin_reg.predict(X_test_significant)
rmse = np.sqrt(mean_squared_error(df_test['target'], predictions))

print("Selected significant features:", significant_features)
print("RMSE on testing dataset:", rmse)


Normality assumption not met for residuals
Confidence Intervals:
                  0            1
const   145.637907   157.053302
age     -97.923288   173.731331
sex    -376.835633  -107.093092
bmi     391.062323   693.795194
bp      207.350409   488.057279
s1    -1818.843994   -44.133698
s2     -198.122152  1234.246706
s3     -294.901155   621.741121
s4      -89.349387   639.985190
s5      357.689156  1114.708562
s6      -95.771349   193.112664
Selected significant features: ['sex', 'bmi', 'bp', 's1', 's5']
RMSE on testing dataset: 53.4335953887283


## Task 3

In [4]:
data = load_wine(as_frame=True)
df = data['frame']

X = df['color_intensity']
Y = df['hue']
Z = df['flavanoids']

### 1

In [5]:
rho_xy, p_value_xy = stats.pearsonr(X, Y)
print(f"Pearson correlation between X and Y: {rho_xy}, p-value: {p_value_xy}")

Pearson correlation between X and Y: -0.5218131932287577, p-value: 8.075008429978309e-14


### 2

In [6]:
mean_X, mean_Y = np.mean(X), np.mean(Y)
std_X, std_Y = np.std(X, ddof=1), np.std(Y, ddof=1)
rho_manual = np.mean((X - mean_X) * (Y - mean_Y)) / (std_X * std_Y)
n = len(X)
t_value = rho_manual * np.sqrt((n - 2) / (1 - rho_manual**2))
p_value_manual = 2 * (1 - stats.t.cdf(np.abs(t_value), df=n-2))
print(f"Manual Pearson correlation: {rho_manual}, t-value: {t_value}, p-value: {p_value_manual}")

Manual Pearson correlation: -0.5188816584353376, t-value: -8.05261114076038, p-value: 1.1746159600534156e-13


### 3

In [7]:
rho_s_xz, p_value_s_xz = stats.spearmanr(X, Z)
print(f"Spearman's correlation between X and Z: {rho_s_xz}, p-value: {p_value_s_xz}")

Spearman's correlation between X and Z: -0.04291038821273014, p-value: 0.5695430180550238


### 4

In [8]:
tau_xz, p_value_tau_xz = stats.kendalltau(X, Z)
print(f"Kendall's tau between X and Z: {tau_xz}, p-value: {p_value_tau_xz}")

Kendall's tau between X and Z: 0.028674293665247572, p-value: 0.5712785725826517
